In [1]:
import os,sys
import astropy.units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.time import Time
import numpy as np

import warnings
from matplotlib import pyplot as plt
from matplotlib import colors as colors
from astropy.io import fits
from astropy.wcs import WCS, FITSFixedWarning


sys.path.append(os.path.expanduser('~/repos/runawaysearch/src'))
sys.path.append(os.path.expanduser('~/repos/ReipurthBallyProject/src'))

from false_image  import false_image
from utils import obs_dirs

from astropy.table import Table

In [2]:
obs_root = '../data'
objname = 'M8'
dirs = obs_dirs(obs_root, objname)

In [3]:
obs_path = os.path.join(dirs['no_bias'], 'SUPA01564806.fits')
with fits.open(obs_path) as hdul:
    hdr = hdul[0].header.copy()
detector = hdr['DETECTOR']

xmatchpath = os.path.join(dirs['xmatch_tables'], detector)
xmatch_tbl = Table.read(xmatchpath)

In [4]:
t_obs = Time(hdr['MJD'], scale='utc', format='mjd')
#hard code for gaia dr3:
t_gaia = Time(2016, scale='tcb',format='jyear')

coords_gaia = SkyCoord(ra = xmatch_tbl['ra']*u.degree, dec = xmatch_tbl['dec']*u.degree,
                pm_ra_cosdec = xmatch_tbl['pmra'].filled(fill_value=0.0)*u.mas/u.year,
                pm_dec = xmatch_tbl['pmdec'].filled(fill_value=0.0)*u.mas/u.year,
                radial_velocity = xmatch_tbl['radial_velocity'].filled(fill_value=0.0)*u.km/u.second,
                distance = 1000.0/np.abs(xmatch_tbl['parallax']) * u.pc,
                obstime = t_gaia)

#move the positions to the obs time and reframe to FK5
t_2000 = Time('J2000.0',  format='jyear_str')
coords = coords_gaia.apply_space_motion(new_obstime=t_2000).fk5

c:\Users\Kevin\anaconda3\envs\astro\lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 80 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


(https://dc.zah.uni-heidelberg.de/tableinfo/gaia.dr2epochflux)

mag = -2.5 log10(flux)+zero point,
where the zero points assumed for Gaia DR2 are 25.6884±0.0018 in G, 25.3514±0.0014 in BP, and 24.7619±0.0019 in RP (VEGAMAG).

25.6874 for DR3 G-band. See [this GAIA DR3 Doc](https://gea.esac.esa.int/archive/documentation/GDR3/Data_processing/chap_cu5pho/cu5pho_sec_photProc/cu5pho_ssec_photCal.html#Ch5.T4)

In [5]:
scale=20
outpath = f'../data/testfits_{scale:.0f}.fits'
phdu = false_image(hdr, coords, xmatch_tbl['phot_g_mean_flux'], scale=scale)
phdu.writeto(outpath, overwrite=True)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [ ]:
def mag_to_flux(mags, zp = 25.6874):
    expnts = (mags-zp)/-2.5
    flux = np.power(10, expnts)
    return flux

In [ ]:
mag_to_flux(18.0)

In [ ]:
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(121,)
zz = ax.hist(mag_to_flux(xmatch_tbl['rMeanPSFMag']), range=(0.0, 20000), bins=50, log=True)

ax = fig.add_subplot(122,)
zz = ax.hist(xmatch_tbl['rMeanPSFMag'], bins=50)



In [ ]:
from scipy.stats import linregress

def add_trend_to_scatter(ax, x, y):
    """
    x,y masked arrays
    """
    valid = np.logical_not(np.logical_or(x.mask, y.mask))
    xy = np.array([x[valid],y[valid]])

    res = linregress(xy)

    xx = np.linspace(x.min(), x.max(), 10000)

    yhat = res.intercept + res.slope*xx

    ax.plot(xx,yhat, color='red', lw=3, label='Fitted')

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111,)
flux_hat = mag_to_flux(xmatch_tbl['phot_g_mean_mag'])
ax.scatter( xmatch_tbl['phot_g_mean_mag'], xmatch_tbl['phot_g_mean_flux'],  s=3, label='Actual Flux', color='green')
ax.scatter( xmatch_tbl['phot_g_mean_mag'], flux_hat,  s=3, label='Predicted Flux', color='red')
#add_trend_to_scatter(ax, flux_hat, xmatch_tbl['phot_g_mean_flux'])
# ax.set_xlim(0, 100000)
#ax.set_ylim(0, 100000)
ax.grid()
ax.legend()
ax.set_xlabel('Observed Magnitude')
ax.set_ylabel('Flux')

In [ ]:
def rmse(hat, act):
    e2 = (hat-act)**2
    return np.sqrt(np.nanmean(e2))

In [ ]:
err = rmse(mag_to_flux(xmatch_tbl['phot_g_mean_mag']),xmatch_tbl['phot_g_mean_flux'] )
print(f'RMSE flux from magnnitude: {err:.4f} flux units')
rel = rmse(mag_to_flux(xmatch_tbl['phot_g_mean_mag']),xmatch_tbl['phot_g_mean_flux'] )/xmatch_tbl['phot_g_mean_flux'].mean()
print(f'Fractional RMSE: {rel:.3e}')

In [ ]:
from scipy.ndimage.filters import gaussian_filter
def false_image(hdu, coords, mags, sigma=3, zeropt = 25.6874, noise_dc= 1, scale=1000):
    
    #fix the magnitude: (missing mag to be considered dimmest)
    f_mags = np.nan_to_num(mags, zeropt)

    s = hdu.data.shape
    wcs = WCS(hdu.header)
 

    in_image = wcs.footprint_contains(coords)
    pxs = wcs.world_to_pixel(coords)

    p_x = np.round(pxs[0][in_image]).astype(int)
    p_y = np.round(pxs[1][in_image]).astype(int)
    f_mags = f_mags[in_image]
    
    img = np.zeros(s, dtype=np.float32)
    img[p_y, p_x] = (zeropt - f_mags)
    img = gaussian_filter(img, sigma=sigma, mode='nearest')*scale

    # # Let's add some noise to the images
    # noise_std = np.sqrt(noise_dc)
    # img += np.random.normal(loc=noise_dc, scale=noise_std, size=img.shape)

    return img

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time

with fits.open(fitspath) as hdul:
    hdu = hdul[0].copy()
    hdr = hdu.header
    wcs = WCS(hdr)

#hard code for gaia dr3:
t_gaia = Time(2016, scale='tcb',format='jyear')
t_obs = Time(hdu.header['MJD'], scale='utc', format='mjd')

coords_gaia = SkyCoord(ra = xmatch_tbl['ra']*u.degree, dec = xmatch_tbl['dec']*u.degree,
                  pm_ra_cosdec = xmatch_tbl['pmra']*u.mas/u.year,
                  pm_dec = xmatch_tbl['pmdec']*u.mas/u.year,
                  radial_velocity = xmatch_tbl['radial_velocity'].filled(fill_value=0.0)*u.km/u.second,
                  distance = 1000.0/np.abs(xmatch_tbl['parallax']) * u.pc,
                  obstime = t_gaia)

#move the positions to the obs time and reframe to FK5
coords = coords_gaia.apply_space_motion(new_obstime=t_obs).fk5

In [ ]:
img = false_image(hdu, coords, xmatch_tbl['rMeanPSFMag'], sigma=10)

In [ ]:
import matplotlib.colors as colors
fig = plt.figure(figsize=(16,12))
ax = fig.add_subplot(121, projection=wcs)
ax.imshow(hdu.data, origin='lower',norm=colors.LogNorm(vmin=1000, vmax=2500), cmap=plt.cm.gray_r)

ax = fig.add_subplot(122, projection=wcs)
ax.imshow(img, origin='lower', cmap=plt.cm.gray_r, norm=colors.LogNorm(vmin=0, vmax=7))


In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection=wcs)
ax.imshow(img, origin='lower', cmap=plt.cm.gray_r, norm=colors.LogNorm(vmin=0, vmax=3))

In [ ]:
img1000= false_image(hdu, coords, xmatch_tbl['rMeanPSFMag'], sigma=10)
img1= false_image(hdu, coords, xmatch_tbl['rMeanPSFMag'], sigma=10, scale=1)

In [ ]:
fig = plt.figure(figsize=(16,12))
ax = fig.add_subplot(121, projection=wcs)
ax.imshow(img1000, origin='lower', cmap=plt.cm.gray_r, norm=colors.LogNorm(vmin=0, vmax=7))
ax.set_title('Scale=1000')

ax = fig.add_subplot(122, projection=wcs)
ax.imshow(img1, origin='lower', cmap=plt.cm.gray_r, norm=colors.LogNorm(vmin=0, vmax=4))
ax.set_title('Scale=1')